In [1]:
# Based on Tensorflow example notebook:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/multilayer_perceptron.ipynb

In [36]:
import numpy as np
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, auc, roc_curve
import matplotlib.pyplot as plt

In [38]:
###load data first
features = np.loadtxt("features_all.txt", delimiter=",")
labels = np.loadtxt("labels_all.txt", delimiter=",")
print(features.shape)
print(labels.shape)
print(features[1:5,1:5])
print(labels[1:5])
X = features
Y = np.array([labels, -(labels-1)]).T  # The model currently needs one column for each class

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(37500, 10)


In [32]:
# Parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 100
display_step = 1


# Network Parameters
n_hidden_1 = 10 # 1st layer number of features
n_hidden_2 = 10 # 2nd layer number of features
n_input = 244 # Number of feature
n_classes = 2 # Number of classes to predict


# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [34]:
# Launch the graph

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X_train)/batch_size)
        X_batches = np.array_split(X_train, total_batch)
        Y_batches = np.array_split(Y_train, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Training Accuracy:", accuracy.eval({x: X_train, y: Y_train}))
    print("Testing Accuracy:", accuracy.eval({x: X_test, y: Y_test}))
    global result 
    result = tf.argmax(pred, 1).eval({x: X_test, y: Y_test})
    result = np.array(result)
    #confusion matrix
    con_mat = confusion_matrix(Y_test[:,1], result)
    print("Confusion Matrix is: ", con_mat)
    
    #ROC curve
    fpr, tpr, threshold = roc_curve(Y_test[:,1], result)
    roc_auc = auc(fpr, tpr)

    # method I: plt

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

Epoch: 0001 cost= 3.590750212
Epoch: 0002 cost= 1.013177722
Epoch: 0003 cost= 0.615347650
Epoch: 0004 cost= 0.431908790
Epoch: 0005 cost= 0.333990873
Epoch: 0006 cost= 0.277345990
Epoch: 0007 cost= 0.241332096
Epoch: 0008 cost= 0.217563164
Epoch: 0009 cost= 0.201451888
Epoch: 0010 cost= 0.190098237
Epoch: 0011 cost= 0.181611979
Epoch: 0012 cost= 0.174924093
Epoch: 0013 cost= 0.169319726
Epoch: 0014 cost= 0.164511786
Epoch: 0015 cost= 0.160508363
Epoch: 0016 cost= 0.156730352
Epoch: 0017 cost= 0.152996223
Epoch: 0018 cost= 0.149623444
Epoch: 0019 cost= 0.146603819
Epoch: 0020 cost= 0.143606648
Epoch: 0021 cost= 0.140778002
Epoch: 0022 cost= 0.138073536
Epoch: 0023 cost= 0.135471609
Epoch: 0024 cost= 0.133036461
Epoch: 0025 cost= 0.130838733
Epoch: 0026 cost= 0.128719182
Epoch: 0027 cost= 0.126771072
Epoch: 0028 cost= 0.124933340
Epoch: 0029 cost= 0.123143376
Epoch: 0030 cost= 0.121390156
Epoch: 0031 cost= 0.119773665
Epoch: 0032 cost= 0.118469528
Epoch: 0033 cost= 0.117352215
Epoch: 003